# read

In [1]:
import pandas as pd; pd.set_option('display.max_rows', None)
import numpy as np; np.random.seed(0)
import matplotlib.pyplot as plt
import seaborn as sns

train = pd.read_csv('./data/train_df.csv', encoding='cp949')
test  = pd.read_csv('./data/test_df.csv', encoding='cp949')
sub   = pd.read_csv('./data/sample_submission.csv', encoding='cp949')

In [2]:
train.shape, test.shape, sub.shape

((32000, 6), (4640, 5), (4640, 2))

In [3]:
train.head()

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT
0,0,1129000014045300,5011000220046300,패션의류,상의,3
1,1,1135000009051200,5011000178037300,생활/건강,반려동물,3
2,2,1135000030093100,5011000265091400,패션의류,기타패션의류,9
3,3,1154500002014200,5011000315087400,식품,농산물,10
4,4,1165000021008300,5011000177051200,식품,가공식품,3


In [4]:
test.head()

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM
0,32000,5013000043028400,1165000021097200,식품,농산물
1,32001,5013000044016100,1154500002066400,식품,농산물
2,32002,5013000205030200,4139000102013200,식품,농산물
3,32003,5013000205030200,4221000040093400,식품,농산물
4,32004,5013000268011400,2726000004017100,식품,농산물


# preprocessing

In [20]:
large_label = dict(train.groupby('DL_GD_LCLS_NM').mean().sort_values('INVC_CONT')[['INVC_CONT']].reset_index().values)
mid_label   = dict(train.groupby('DL_GD_MCLS_NM').mean().sort_values('INVC_CONT')[['INVC_CONT']].reset_index().values)

train['DL_GD_LCLS_NM'] = train['DL_GD_LCLS_NM'].map(large_label)
test['DL_GD_LCLS_NM']  = test['DL_GD_LCLS_NM'].map(large_label)

train['DL_GD_MCLS_NM'] = train['DL_GD_MCLS_NM'].map(mid_label)
test['DL_GD_MCLS_NM']  = test['DL_GD_MCLS_NM'].map(mid_label)

In [21]:
train.head()

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT
0,0,1129000014045300,5011000220046300,6.678694,3.672897,3
1,1,1135000009051200,5011000178037300,5.407921,4.444134,3
2,2,1135000030093100,5011000265091400,6.678694,10.375479,9
3,3,1154500002014200,5011000315087400,4.658195,4.297401,10
4,4,1165000021008300,5011000177051200,4.658195,6.807151,3


In [22]:
# 격자공간고유번호 슬라이싱

for i in [3, 4, 5, 6, 7, 8, 9]:
    train['send_' + str(i)] = train['SEND_SPG_INNB'].astype('str').str[:i].astype('int64')
    test['send_' + str(i)]  = test['SEND_SPG_INNB'].astype('str').str[:i].astype('int64')
    
    train['rec_' + str(i)] = train['REC_SPG_INNB'].astype('str').str[:i].astype('int64')
    test['rec_' + str(i)]  = test['REC_SPG_INNB'].astype('str').str[:i].astype('int64')

In [23]:
train.head()

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT,send_3,rec_3,send_4,rec_4,send_5,rec_5,send_6,rec_6,send_7,rec_7,send_8,rec_8,send_9,rec_9
0,0,1129000014045300,5011000220046300,6.678694,3.672897,3,112,501,1129,5011,11290,50110,112900,501100,1129000,5011000,11290000,50110002,112900001,501100022
1,1,1135000009051200,5011000178037300,5.407921,4.444134,3,113,501,1135,5011,11350,50110,113500,501100,1135000,5011000,11350000,50110001,113500000,501100017
2,2,1135000030093100,5011000265091400,6.678694,10.375479,9,113,501,1135,5011,11350,50110,113500,501100,1135000,5011000,11350000,50110002,113500003,501100026
3,3,1154500002014200,5011000315087400,4.658195,4.297401,10,115,501,1154,5011,11545,50110,115450,501100,1154500,5011000,11545000,50110003,115450000,501100031
4,4,1165000021008300,5011000177051200,4.658195,6.807151,3,116,501,1165,5011,11650,50110,116500,501100,1165000,5011000,11650000,50110001,116500002,501100017


# encoding

In [24]:
train.drop(['index', 'SEND_SPG_INNB', 'REC_SPG_INNB'], axis=1, inplace=True)
test.drop(['index', 'SEND_SPG_INNB', 'REC_SPG_INNB'], axis=1, inplace=True)

In [25]:
train.head()

,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT,send_3,rec_3,send_4,rec_4,send_5,rec_5,send_6,rec_6,send_7,rec_7,send_8,rec_8,send_9,rec_9
0,6.678694,3.672897,3,112,501,1129,5011,11290,50110,112900,501100,1129000,5011000,11290000,50110002,112900001,501100022
1,5.407921,4.444134,3,113,501,1135,5011,11350,50110,113500,501100,1135000,5011000,11350000,50110001,113500000,501100017
2,6.678694,10.375479,9,113,501,1135,5011,11350,50110,113500,501100,1135000,5011000,11350000,50110002,113500003,501100026
3,4.658195,4.297401,10,115,501,1154,5011,11545,50110,115450,501100,1154500,5011000,11545000,50110003,115450000,501100031
4,4.658195,6.807151,3,116,501,1165,5011,11650,50110,116500,501100,1165000,5011000,11650000,50110001,116500002,501100017


# modeling

In [26]:
X = train.drop('INVC_CONT', axis=1)
y = train['INVC_CONT']

In [27]:
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV

model = CatBoostRegressor(random_state=0, random_strength=0)

param = {
    
}

gscv = GridSearchCV(
    estimator=model, param_grid=param, scoring='neg_root_mean_squared_error', 
    refit=True, cv=5
)

gscv.fit(X, y)

Learning rate set to 0.068341
0:	learn: 5.4330044	total: 164ms	remaining: 2m 44s
1:	learn: 5.4203162	total: 179ms	remaining: 1m 29s
2:	learn: 5.4092503	total: 195ms	remaining: 1m 4s
3:	learn: 5.3988180	total: 208ms	remaining: 51.7s
4:	learn: 5.3904643	total: 220ms	remaining: 43.7s
5:	learn: 5.3823997	total: 231ms	remaining: 38.2s
6:	learn: 5.3732186	total: 250ms	remaining: 35.4s
7:	learn: 5.3671510	total: 264ms	remaining: 32.8s
8:	learn: 5.3615865	total: 276ms	remaining: 30.4s
9:	learn: 5.3565141	total: 291ms	remaining: 28.8s
10:	learn: 5.3522402	total: 306ms	remaining: 27.5s
11:	learn: 5.3477815	total: 318ms	remaining: 26.1s
12:	learn: 5.3272892	total: 431ms	remaining: 32.7s
13:	learn: 5.3232003	total: 472ms	remaining: 33.2s
14:	learn: 5.3197154	total: 545ms	remaining: 35.8s
15:	learn: 5.3020699	total: 589ms	remaining: 36.2s
16:	learn: 5.2860459	total: 645ms	remaining: 37.3s
17:	learn: 5.2829994	total: 699ms	remaining: 38.1s
18:	learn: 5.2781371	total: 796ms	remaining: 41.1s
19:	learn

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostRegressor object at 0x000001B2BF9D1E20>,
             param_grid={}, scoring='neg_root_mean_squared_error')

In [28]:
print(gscv.best_params_)
print(-gscv.best_score_)

{}
5.634688789661978


In [29]:
pred = gscv.predict(test)
sub['INVC_CONT'] = pred

In [30]:
sub.head()

,index,INVC_CONT
0,32000,4.339059
1,32001,4.650605
2,32002,5.080647
3,32003,5.928550
4,32004,4.644832


In [31]:
# sub.to_csv('./sub/logistics_02.csv', index=False)